<a href="https://colab.research.google.com/github/Ngom/My_Codes_Dsti/blob/main/ANN_MNIST_practice_4_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install not preinstall libraries for colab
!pip install patchify
!pip install tensorflow-gpu

In [ ]:
# import needed libraries
import tensorflow as tf
from tensorflow import keras
from keras import datasets
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# check if GPU is used
my_gpu_device_name = tf.test.gpu_device_name()

if my_gpu_device_name != '/device:GPU:0':
  raise SystemError('GPU not fund')

print(f'Availablabe GPU : {my_gpu_device_name}')

In [ ]:
# check which GPU
qpu_info = !nvidia-smi
gpu_info = '\n'.join(qpu_info)

if gpu_info.find('failed') >= 0:
  print('Select the RunTime > "Change runtime type" to enable a GPU accelerator,')
  print('and, then re-execute this shell')
else: 
  info = qpu_info

info

In [ ]:
# check how much memory is still available
from psutil import virtual_memory 
ram_gb = virtual_memory().total/1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#show the version of TensorFlow we are using
print(tf.__version__)

In [ ]:
# load mnist dataset
mnist = datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# visualize few examples
plt.figure(figsize=(10,2))
for index, (image, label) in enumerate(zip(X_train[:5], y_train[:5])):
  plt.subplot(1,5, index+1)
  plt.imshow(image, cmap=plt.cm.gray)
  plt.title('Training: {}\n'.format(label), fontsize=10)
plt.show()

In [ ]:
# Scling data : convert from integers to floating-point numbers (between 0 and 1) 
X_train, X_test = X_train/255.0, X_test/255.0

In [ ]:
# build the sequential model
model = keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2)) # Regularizers adding on hidden layer
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.08)) # Regularizers adding on hidden layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
from tensorflow.python.eager.monitoring import Metric
# compile the model: optimier, loss, and eval_metrics
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              # 'sparse_categorical_crossentropy' used when the true labels are integers 
              # and 'categorical_crossentropy' is used when the true labels are one-hot encoded 
              metrics=['accuracy']
              )

In [ ]:
print(np.shape(X_train))
print(np.shape(X_test))

In [ ]:
# train
history = model.fit(X_train,
                    y_train,
                    batch_size=32, # default batch_size=32
                    epochs=20
                    )

In [ ]:
# summarize
model.summary()

## Visualize the network

In [ ]:
from keras.utils.vis_utils import plot_model
# visualize the neural network
plot_model(model=model, to_file='/content/model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# evalate 
loss, accuracy = model.evaluate(X_test, y_test)
print("loss = {}, accuracy = {}".format(loss, accuracy))

In [ ]:
# predict
from sklearn import metrics
predictions = model.predict(X_test) # get predicted classes probabilities
print(predictions)

In [ ]:
# to get classes : way 1
# loop coding
prediction = [np.argmax(i) for i in predictions]
print(prediction)

In [ ]:
# to get classes : way 2
# vectorization coding
classes = np.argmax(predictions, axis=1)
print(classes)

In [ ]:
plt.figure(figsize=(9,3))
for index, (image, label) in enumerate(zip(X_test[-10:], classes[-10:])):
  plt.subplot(2, 5, index+1)
  plt.imshow(image, cmap=plt.cm.gray)
  plt.title('Training: {}\n'.format(label), fontsize=10)
plt.show()

In [ ]:
# confusion matrix
import seaborn as sns
cm = metrics.confusion_matrix(y_test, classes, normalize='true')
plt.figure(figsize=(10,10))
sns.heatmap(cm,
            annot=True,
            fmt=".3f",
            linewidths=.5,
            square=True,
            cmap = 'Blues_r',
            )
plt.ylabel('Actual classes')
plt.xlabel('Predicted classes')
all_sample_title = 'Accuracy = {0}'.format(accuracy)
plt.title(all_sample_title, size=15)
plt.show()

In [ ]:
###################### CALLBACKS #################################
# A callback performs actions at various stages of training 
# (e.g. at the start or end of an epoch, before or after a single batch)

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, CSVLogger
import os, datetime

In [ ]:
from numpy.lib.histograms import histogram
from tensorflow.python.ops.gen_batch_ops import batch
from sklearn.utils import validation

# Early Stopping : which quantity to monitor to decide early stopping
#patience = the number of epochs to wait before stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# model checkpoint
model_checkpoint = ModelCheckpoint(filepath='best_model.h5',
                                   monitor='val_loss',
                                   save_best_only=True
                                   )
# set logs dir_path for tensorboard
log_dir = os.path.join("logs", datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
tsnrborab = TensorBoard(log_dir=log_dir, histogram_freq=1)

# set csv path
csv_log_path = "/content/logs.scv"
csv_logger = CSVLogger(csv_log_path,
                       append=True,
                       separator=','
                       )
# callbacks criteria
callbacks = [early_stopping,
             model_checkpoint,
             tsnrborab,
             csv_logger
             ]
# build the model and get history
history = model.fit(X_train,
                    y_train,
                    epochs=50,
                    validation_split=0.25,
                    batch_size=40,
                    verbose=2,
                    callbacks = [callbacks]
                    )

In [ ]:
# Savee model can be reloaded
saved_model =  keras.models.load_model('best_model.h5')
# view the best
saved_model.summary()

In [ ]:
test_loss, test_acc = saved_model.evaluate(X_test, y_test)
print(f'Accuracy of best model = {test_acc}')

In [ ]:
# view your tensorboard
%load_ext tensorboard
%tensorboard --logdir logs

1. 2-D CNN VISUALIZATION: 
[click on this link](https://www.cs.cmu.edu/~aharley/nn_vis/cnn/2d.html)

2. DATA AUGMENTATION:
[check this url](https://www.tensorflow.org/tutorials/images/data_augmentation)

3. IMAGES DATA GENERATOR:
[check this page](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

4.

In [ ]:
'''#Regularizers :
from tensorflow.keras.regularizers import l2, l1
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(4, activation='relu', kernel_regularizer=l1(l=0.1)))
model.add(tf.keras.layers.Dense(4, activation='relu', kernel_regularizer=tensorflow.keras.regularizers.l1_l2(l1=0.1, l2=0.01)))
'''